In [4]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
# from cnn_utils import *

%matplotlib inline
np.random.seed(1)

In [5]:
import json
import random

In [ ]:
tf.nn.softmax_cross_entropy_with_logits

In [ ]:
EPOCHS = 10
BATCH_SIZE = 32
IMG_SIZE = 227
data_file = "dataset.txt"

In [3]:
def conv_layer(input_data,channels_in, filter_size,num_filters,stride_size,pad="SAME",name="conv",weight_name="W1"):
    with tf.name_scope(name):
        W=tf.get_variable(weight_name,shape=[filter_size,filter_size,channels_in,num_filters],initializer=tf.contrib.layers.xavier_initializer(seed=0))
        conv = tf.nn.conv2d(input_data,W,strides=[1,stride_size,stride_size,1],padding=pad)
        act = tf.nn.relu(conv)
        tf.summary.histogram("weights", W)
        tf.summary.histogram("activations", act)
        return act

In [8]:

def alexnet_model(input_data,image_channels):
    #input_size = [227*227*3]
    conv1 = conv_layer(input_data,image_channels,11,96,4,"VALID","conv1","W1")
    #output size = [55*55*96]
    P1 = tf.nn.max_pool(conv1,ksize=[1,3,3,1], strides=[1,2,2,1],padding="VALID")
    #output_size = [27*27*96]
    
    conv2 = conv_layer(P1,96,5,256,1,"SAME","conv2","W2")
    #output_size = [27*27*256]
    P2 = tf.nn.max_pool(conv2,ksize=[1,3,3,1], strides=[1,2,2,1],padding="VALID")
    #output_size = [13*13*256]
    
    conv3 = conv_layer(P2,256,3,384,1,"SAME","conv3","W3")
    #output_size = [13*13*384]
    
    conv4 = conv_layer(conv3,384,3,384,1,"SAME","conv4","W4")
    #output_size = [13*13*384]
    
    conv5 = conv_layer(conv4,384,3,256,1,"SAME","conv5","W5")
    #output_size = [13*13*256]
    
    P3 = tf.nn.max_pool(conv5,ksize=[1,3,3,1], strides=[1,2,2,1],padding="VALID")
    #output_size = [6*6*256]
    
    P3 = tf.contrib.layers.flatten(P3)
    
    fc6 = tf.contrib.layers.fully_connected(P3,4096,scope="fc6")
    tf.summary.histogram("fc6/relu", fc6)
    
    dropout1 = tf.layers.dropout(inputs=fc6, rate=0.5)
    
    fc7 = tf.contrib.layers.fully_connected(dropout1,4096,scope="fc7")
    tf.summary.histogram("fc7/relu", fc7)
    
    dropout2 = tf.layers.dropout(inputs=fc7, rate=0.5)
    
    fc8 = tf.contrib.layers.fully_connected(dropout2,2,activation_fn=None, scope="fc8")
    tf.summary.histogram("fc8", fc8)
    
    return fc8

In [ ]:
def prepare_data(data):
    newData=[]
    labels=[]
    print("preparing data....")
    for sample in data:
        img_path,label = sample.strip().split(" ")
        
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        
        img[:, :, 0] = cv2.equalizeHist(img[:, :, 0])
        img[:, :, 1] = cv2.equalizeHist(img[:, :, 1])
        img[:, :, 2] = cv2.equalizeHist(img[:, :, 2])

        #Image Resizing
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation = cv2.INTER_CUBIC)
        
        #img = Image.open(img_path)
        #img = img.resize((IMG_SIZE,IMG_SIZE))
        img = np.array(img)
        img = img/255
        newData.append(img)
        if "cat" in img_path:
            labels.append(np.array([1,0]))
        elif "dog" in img_path:
            labels.append(np.array([0,1]))
    print("preparing data completed.")
    return np.array(newData),np.array(labels)

In [ ]:
f = open(data_file,"r")
data = f.readlines()
random.shuffle(data)
print(len(data))
train_data = data[:20000]
val_data = data[20000:]

In [ ]:
tf.reset_default_graph()
learning_rate = 0.001

with tf.name_scope('inputs'):
    X = tf.placeholder(tf.float32, shape=(None,IMG_SIZE,IMG_SIZE , 3),name="X")
    tf.summary.image('input', X, BATCH_SIZE)
    Y = tf.placeholder(tf.float32,shape=(None,2),name="labels")
    
logits = vggnet(X)
with tf.name_scope("loss"):    
    train_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y), name="loss")
    tf.summary.scalar("loss", train_loss)

with tf.name_scope("train"):
    #optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(train_loss)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(train_loss)
print("optimizer set")

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)
print("accuracy set")

merged_summary = tf.summary.merge_all()

# init = 
# print("initializing the variables")

with tf.Session() as sess:
    print("session created.")
    sess.run(tf.global_variables_initializer())
    
    print("writing tensorboard")
    writer = tf.summary.FileWriter("/tmp/tensorboard/cat-dog-vgg1")
    writer.add_graph(sess.graph)
    print("writer added to graph")
    
    for epoch in range(EPOCHS):
        
        print("processing epoch "+str(epoch))
        print("total number of batches in 1 epoch: "+ str(len(train_data)/BATCH_SIZE))
        
        for i in range(0,len(train_data),BATCH_SIZE): 
#             print(i)
            print("Batch:" + str((i/BATCH_SIZE)+1))
            
            minibatch_X, minibatch_Y = prepare_data(train_data[i:i+BATCH_SIZE])
            
            if i % (BATCH_SIZE*4) == 0:  # Record summaries and test-set accuracy
                summary = sess.run(merged_summary, feed_dict={X:minibatch_X,Y:minibatch_Y})
                writer.add_summary(summary, i + epoch*len(train_data))
            
            #sess.run(optimizer,feed_dict={X:minibatch_X,Y:minibatch_Y})
            _ , temp_loss, temp_accuracy = sess.run([optimizer,train_loss,accuracy],feed_dict={X:minibatch_X,Y:minibatch_Y})
    
            #writer.add_summary(summary, i)
            print("temp_loss: "+str(temp_loss))
            
#             if i % (BATCH_SIZE*8) == 0 and i!=0:
#                 print("validating...")
#                 for v in range(0,len(val_data),50):
#                     val_x,val_y = prepare_data(val_data[v:v+BATCH_SIZE])
#                     summary = sess.run(merged_summary, feed_dict={X:val_x, Y:val_y})
#                     writer.add_summary(summary, v + epoch*len(val_data))
            
            
#             print("temp_accuracy: "+str(temp_accuracy))
#     train_accuracy = accuracy.eval({X: train_data, Y: train_labels})
#     val_data, val_labels = prepare_data(val_data)
#     val_accuracy = accuracy.eval({X: val_data, Y: val_labels})
# #     print("Train Accuracy:", train_accuracy)
#     print("Validation Accuracy:", val_accuracy)

In [9]:
tf.reset_default_graph()
learning_rate = 0.009
input_channels = 3
num_epochs= 20
minibatch_size = 64
with tf.name_scope('inputs'):
    X = tf.placeholder(tf.float32, shape=(None,227, 227, 3),name="X")
    Y = tf.placeholder(tf.float32,shape=(None,1000),name="labels")

logits = alexnet_model(X,input_channels)

with tf.name_scope("loss"):    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=Y), name="loss")
    tf.summary.scalar("loss", loss)

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)

summ = tf.summary.merge_all()

init = tf.global_variables_initializer()


with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter("/tmp/test-tensorboard/1")
    writer.add_graph(sess.graph)
    for epoch in range(num_epochs):
        minibatch_cost = 0.
        num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
        seed = seed + 1
        minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

        for minibatch in minibatches:

            # Select a minibatch
            (minibatch_X, minibatch_Y) = minibatch
            # IMPORTANT: The line that runs the graph on a minibatch.
            # Run the session to execute the optimizer and the cost, the feedict should contain a minibatch for (X,Y).
            ### START CODE HERE ### (1 line)
            _ , temp_loss, temp_accuracy = sess.run([optimizer,loss,accuracy],feed_dict={X:minibatch_X,Y:minibatch_Y})
            ### END CODE HERE ###

            minibatch_cost += temp_loss / num_minibatches

        
        # Print the cost every epoch
        if print_cost == True and epoch % 5 == 0:
            print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
        if print_cost == True and epoch % 1 == 0:
            costs.append(minibatch_cost)
    
    
    
    
    
    
    

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



NameError: name 'm' is not defined